In [1]:
!pip install happybase

In [2]:
import happybase
import pandas as pd

In [4]:
con = happybase.Connection("localhost", 9090)

In [21]:
t = con.table("test_dummy")

In [23]:
def read_df(analysis_name, partition_key, table = "test_dummy"):
    """Reads from HBase spark analysis output table"""
    count = 0
    while count<10:
        try:
            con = happybase.Connection("localhost", 9090)
            t = con.table(table)
            break
        except:
            count += 1
            if count == 9:
                print("Connection failure!")
            pass
    
    df = pd.read_json(t.row(str.encode(partition_key))[str.encode(f'df:{analysis_name}')].decode("utf-8"), lines=True)
    return df

In [24]:
res = read_df("dummy", "2022-01-15-13")

In [25]:
res.head()

,VehicleNumber,count
0,7253,243
1,9427,243
2,9852,243
3,1025,243
4,1507,243
